In [7]:
# -*- coding: utf-8 -*-
"""
Created on Sun Aug 16 17:10:53 2020

@author: wanxiang.shen@u.nus.edu
"""

import warnings, os
warnings.filterwarnings("ignore")


import pandas as pd
import numpy as np
from glob import glob
from joblib import load, dump
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_auc_score
from sklearn.metrics import auc as calculate_auc

import matplotlib.pyplot as plt
import seaborn as sns

from aggmap import AggMap, AggModel, loadmap
from aggmap import show
np.random.seed(666) #just for reaptable results

import tensorflow as tf
import os
os.environ["CUDA_VISIBLE_DEVICES"]="5"

sns.set(style= 'white',font_scale = 1.2)

In [2]:
flist = glob('../data/*.csv.gzip')
flist = pd.Series(flist).sort_values().tolist()
fall = []
for i in flist:
    df1 = pd.read_csv(i, compression='gzip', index_col = 0)
    df1['class'] = i.split('/')[-1].split('.csv')[0]
    fall.append(df1)
    
df = pd.concat(fall, axis=0)

In [3]:
dfx = df[df.columns[:-1]]
dfy = df[df.columns[-1:]]
dfx = np.log2(dfx + 1) #apply log2(x + 1)


data_save_folder = '/raid/shenwanxiang/transcriptome/pan-cancer'
mp = loadmap('/raid/shenwanxiang/agg_mp_object/pan-cancer.mp')


X_ns = load(os.path.join(data_save_folder, 'RP1_noisys.data')) 
X = X_ns[0]
Y = pd.get_dummies(dfy['class']).values

In [5]:
dfy['idx'] = range(len(dfy))

In [8]:
res5 = []

stddevs = np.arange(0, 0.5, 0.08)

for X, std in zip(X_ns, stddevs):
    
    for i in range(10):

        fold_num = "fold_%s" % str(i).zfill(2) 
        save_path = './fold_results_c1/%s' % fold_num
        
        test_index = pd.read_csv(os.path.join(save_path, 'test_true_label.csv'), index_col=0).index
        test_idx = dfy[dfy.index.isin(test_index)]['idx'].tolist()
        train_idx = dfy[~dfy.index.isin(test_index)]['idx'].tolist()

        testY = Y[test_idx]
        testX = X[test_idx]

        trainX = X[train_idx]
        trainY = Y[train_idx]
        
        model = tf.keras.models.load_model(os.path.join(save_path, 'model.h5'))
        ls, acc = model.evaluate(testX, testY, verbose = 0)
        res5.append([std, fold_num, ls, acc])
    

In [9]:
df5 = pd.DataFrame(res5, columns=['std', 'fold_num', 'loss', 'acc'])

In [11]:
df5.groupby('std').mean().round(3)

,loss,acc
std,,
0.00,0.381,0.951
0.08,0.436,0.940
0.16,0.859,0.864
0.24,2.589,0.627
0.32,5.917,0.364
0.40,9.361,0.209
0.48,12.125,0.132
